In [6]:
import os
import json
import uuid
from typing import List, Tuple, Literal, Annotated

from dotenv import load_dotenv
from langchain.chains.summarize.map_reduce_prompt import prompt_template
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage
# from langchain_core.pydantic_v1 import BaseModel
from pydantic import BaseModel
# from langgraph.graph import StateGraph, END, START
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
# import gradio as gr

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client("gen-ai-hub")

load_dotenv()
aicore_home = os.getenv("AICORE_HOME")
profile = os.getenv("AICORE_PROFILE", "default")
config_path = os.path.join(aicore_home, f"config.json")
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# 구성 파일에서 값 읽어오기
with open(config_path, "r") as config_file:
    config = json.load(config_file)

# 클라이언트 초기화
ai_core_client = AICoreV2Client(
    base_url=config["AICORE_BASE_URL"],
    auth_url=config["AICORE_AUTH_URL"],
    client_id=config["AICORE_CLIENT_ID"],
    client_secret=config["AICORE_CLIENT_SECRET"],
)

print("AI Core client initialized successfully!")
llm = ChatOpenAI(proxy_model_name='gpt-4o-mini', proxy_client=proxy_client)
model = ChatOpenAI(proxy_model_name='gpt-4o-mini', proxy_client=proxy_client)
from langchain_core.prompts import PromptTemplate


AI Core client initialized successfully!


In [7]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI

# Assume OPENAI_API_KEY is set
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
# model = ChatOpenAI()

# This creates a RunnableSequence instance
chain = prompt | model
response = chain.invoke({"topic": "cats"})
print(response.content)

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


In [10]:
template02 = """ Based on the following information, summarize all the working hours and calculate compensatory leave hours. 
    Follow these steps very carefully:
        
    1. Set fixed Over Time standards:
       Weekday fixed Over Time = 30 hours
       Weekend fixed Over Time = 10 hours
    
    2. Calculate weekday compensatory leave:
       a. Excess weekday overtime = MAX(0, weekday_overtime - 30)
       b-1. no exceeding the weekday fixed over time, Weekday compensatory leave = (Excess weekday overtime × 1.5) + (Weekday night work × 0.5)
       b-2. exceeding the weekday fixed over time, Weekday compensatory leave = (Excess weekday overtime × 1.5) + (Weekday night work × 2)
    
    3. Calculate weekend compensatory leave:
       a. Excess weekend overtime = MAX(0, weekend_overtime - 10)
       b-1. If no exceeding the weekend fixed over time, Weekend compensatory leave = (excess weekend overtime × 1.5) + (Weekend night work × 0.5) 
       b-2. If exceeding the weekend fixed over time, Weekend compensatory leave = (excess weekend overtime × 1.5) + (Weekend night work × 2)
    
    4. Calculate total compensatory leave:
       Total compensatory leave hours = Weekday compensatory leave + Weekend compensatory leave
    
    5. Round all calculated hours to two decimal places.provide a clear and concise summary of the overtime hours they want to calculate for compensatory leave.
    
    output format summary as follows:
    <summary>
    === 보상 휴가 시간 계산 결과 ===
    평일 보상 휴가 시간: 
    주말/휴일 보상 휴가 시간: 
    총 보상 휴가 시간: 
    주말/휴일 근무 시간에 "대체 휴일"을 적용한 경우, 주말/휴일 보상 시간은 위의 계산 결과와 다를 수 있습니다.
    </summary>
    
    {reqs}

"""
prompt = ChatPromptTemplate.from_template(
    template=template02
)
reqs = "평일 연장근무는 몇시간 인가요? 10시간입니다. 평일 야간근무는 0시간 입니다. 주말 연장근무는 30분이구요. 주말 야간근무는 4시간입니다. 보상휴가를 시간을 계산해라 "
chain1 = prompt | llm  
response=chain1.invoke({"reqs": reqs})
print(response.content)


To calculate the compensatory leave hours based on the provided information, we will follow the outlined steps.

1. **Fixed Over Time Standards:**
   - Weekday fixed Over Time: 30 hours
   - Weekend fixed Over Time: 10 hours

2. **Given Data:**
   - Weekday overtime: 10 hours
   - Weekday night work: 0 hours
   - Weekend overtime: 0.5 hours (30 minutes)
   - Weekend night work: 4 hours

3. **Calculate Weekday Compensatory Leave:**
   a. Excess weekday overtime = MAX(0, weekday_overtime - 30)
   - Excess weekday overtime = MAX(0, 10 - 30) = 0 hours (no excess)

   b. Since there is no exceeding the weekday fixed overtime, we use the first formula for compensatory leave:
   - Weekday compensatory leave = (0 × 1.5) + (0 × 0.5) = 0 hours

4. **Calculate Weekend Compensatory Leave:**
   a. Excess weekend overtime = MAX(0, weekend_overtime - 10)
   - Excess weekend overtime = MAX(0, 0.5 - 10) = 0 hours (no excess)

   b. Since there is no exceeding the weekend fixed overtime, we use the firs